In [1]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import warnings
import eli5
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\searg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv", encoding="utf8", sep='\t')

In [3]:
df = df[df["sentiment"]!='neautral']

In [4]:
df['review_processed'] = df['review'].progress_apply(lambda x: re.sub(r'[^\w\s]', '', x)).values
df['review_processed'] = df['review_processed'].progress_apply(lambda x: x.lower())
df['review_processed'] = df['review_processed'].progress_apply(lambda x: word_tokenize(x))
df

100%|██████████| 60000/60000 [00:10<00:00, 5637.62it/s]


,review,sentiment,review_processed
0,качество плохое пошив ужасный (горловина напер...,negative,"[качество, плохое, пошив, ужасный, горловина, ..."
1,"Товар отдали другому человеку, я не получила п...",negative,"[товар, отдали, другому, человеку, я, не, полу..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[ужасная, синтетика, тонкая, ничего, общего, с..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"[товар, не, пришел, продавец, продлил, защиту,..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"[кофточка, голая, синтетика, носить, не, возмо..."
...,...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive,"[сделано, достаточно, хорошо, на, ткани, сдела..."
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive,"[накидка, шикарная, спасибо, большое, провдо, ..."
89997,спасибо большое ) продовца рекомендую.. заказа...,positive,"[спасибо, большое, продовца, рекомендую, заказ..."
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive,"[очень, довольна, заказом, меньше, месяца, в, ..."


In [5]:
morph = pymorphy2.MorphAnalyzer()
df['review_morphed'] = df['review_processed'].progress_apply(lambda x: [morph.parse(word)[0].normal_form for word in x])
df

100%|██████████| 60000/60000 [05:23<00:00, 185.64it/s]


,review,sentiment,review_processed,review_morphed
0,качество плохое пошив ужасный (горловина напер...,negative,"[качество, плохое, пошив, ужасный, горловина, ...","[качество, плохой, пошив, ужасный, горловина, ..."
1,"Товар отдали другому человеку, я не получила п...",negative,"[товар, отдали, другому, человеку, я, не, полу...","[товар, отдать, другой, человек, я, не, получи..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[ужасная, синтетика, тонкая, ничего, общего, с...","[ужасный, синтетик, тонкий, ничего, общий, с, ..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"[товар, не, пришел, продавец, продлил, защиту,...","[товар, не, прийти, продавец, продлить, защита..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"[кофточка, голая, синтетика, носить, не, возмо...","[кофточка, голый, синтетик, носить, не, возможно]"
...,...,...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive,"[сделано, достаточно, хорошо, на, ткани, сдела...","[сделать, достаточно, хорошо, на, ткань, сдела..."
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive,"[накидка, шикарная, спасибо, большое, провдо, ...","[накидка, шикарный, спасибо, большой, провдо, ..."
89997,спасибо большое ) продовца рекомендую.. заказа...,positive,"[спасибо, большое, продовца, рекомендую, заказ...","[спасибо, большой, продовца, рекомендовать, за..."
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive,"[очень, довольна, заказом, меньше, месяца, в, ...","[очень, довольный, заказ, маленький, месяц, в,..."


In [6]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform([" ".join(x) for x in df['review_morphed']])
X.shape

(60000, 396100)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

reg = LogisticRegression()
reg.fit(X_train, Y_train)
predict = reg.predict_log_proba(X_test)[:, 1]

roc_auc_score(Y_test, predict)

0.9777779791573166

In [8]:
eli5.show_weights(estimator=reg, 
                  feature_names=list(vectorizer.get_feature_names_out()),
                  top=(10, 10))

Weight?,Feature
+10.460,отличный
+10.075,хороший
+7.917,супер
+7.339,спасибо
+7.282,немного
+6.904,хорошо
+6.754,отлично
+6.230,приятный
+6.177,классный
+6.061,довольный


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

parameters = {'C':[0.5, 1, 2, 5, 10],
              'max_iter':[10, 50, 100],
              'penalty':['none', 'l1', 'l2']}
reg = LogisticRegression()
clf = GridSearchCV(reg, parameters, verbose=10)

clf.fit(X_train, Y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV 1/5; 1/45] START C=0.5, max_iter=10, penalty=none...........................
[CV 1/5; 1/45] END C=0.5, max_iter=10, penalty=none;, score=0.923 total time=   1.2s
[CV 2/5; 1/45] START C=0.5, max_iter=10, penalty=none...........................
[CV 2/5; 1/45] END C=0.5, max_iter=10, penalty=none;, score=0.920 total time=   1.3s
[CV 3/5; 1/45] START C=0.5, max_iter=10, penalty=none...........................
[CV 3/5; 1/45] END C=0.5, max_iter=10, penalty=none;, score=0.921 total time=   1.2s
[CV 4/5; 1/45] START C=0.5, max_iter=10, penalty=none...........................
[CV 4/5; 1/45] END C=0.5, max_iter=10, penalty=none;, score=0.924 total time=   1.4s
[CV 5/5; 1/45] START C=0.5, max_iter=10, penalty=none...........................
[CV 5/5; 1/45] END C=0.5, max_iter=10, penalty=none;, score=0.924 total time=   1.2s
[CV 1/5; 2/45] START C=0.5, max_iter=10, penalty=l1.............................
[CV 1/5; 2/45] END C=0.5, m

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.5, 1, 2, 5, 10], 'max_iter': [10, 50, 100],
                         'penalty': ['none', 'l1', 'l2']},
             verbose=10)

In [ ]:
clf.best_params_

{'C': 0.5, 'max_iter': 10, 'penalty': 'l1'}

In [37]:
reg = LogisticRegression(C = 1, max_iter=100, penalty='elasticnet', solver="saga", l1_ratio=0)
reg.fit(X_train, Y_train)
predict = reg.predict_log_proba(X_test)[:, 1]

roc_auc_score(Y_test, predict)

0.9777782631189277